In [1]:
import numpy as np
import pandas as pd
import signal
from random import sample
import tensorflow as tf
import os, time, json
from math import asin
from random import choice, choices
from shutil import copyfile

def loadArrayInt(fn, usecols=None):
    return np.array(pd.read_csv(fn, sep=' ', header=None, usecols=usecols, dtype=np.int32))
def save_csv(fn, data):
    pd.DataFrame(data).to_csv(fn, sep=' ', header=0, index=False)

def read_node(fn):
    with open(fn, 'r') as fp:
        n_node, n_edge = [int(x) for x in fp.readline()[:-1].split(' ')[:2]]
    edges = np.array(pd.read_csv(fn, sep=' ', header=None, skiprows=1, dtype=np.int32))
    print(n_node, n_edge)
    assert(n_edge == edges.shape[0])
    return edges, n_node
def partDict(parts_):
    part_dict, cnt = {}, 0
    for part_ in parts_:
        if(not part_ in part_dict):
            part_dict[part_] = cnt
            cnt += 1
    parts_ = np.array([part_dict[part_] for part_ in parts_])
#     print("get parts", len(part_dict), np.max(parts_))
    return parts_

2025-10-23 13:55:22.332642: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-23 13:55:22.333975: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-23 13:55:22.337833: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-23 13:55:22.347905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761195322.364745 3572790 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761195322.36

## Configure

In [2]:
RN, F_man_dim = "BJ", 64
F_n_train, F_n_val = int(1e7), int(1e6)
F_batch_size, F_learning_rate = 4096, 4e-2

SaveEmb, n_part_left = 1, 8
dir_road, dir_data = "./data/" + RN + '/', "./train/" + RN + '/'
f_edge, = [dir_road+x for x in [RN+'.gr']]
f_part_full = dir_road+"Nodes_full_4_%d.data"%n_part_left
F_train_dir, f_log, f_stat = dir_data+"model/", dir_data+"log.out", dir_data+"stat"
f_train, f_test = [dir_data + "%s.data"%x for x in ['train', 'test']]
f_emb_output = dir_road + "emb%d_"%(F_man_dim)
if(not os.path.exists(F_train_dir)):
    os.mkdir(F_train_dir)

## Load Files

In [3]:
edges_g, n_node = read_node(f_edge)
nodes = np.zeros((n_node, 2))
parts = loadArrayInt(f_part_full)[:, 1::2]
print("nodes:", nodes.shape, "edges:", edges_g.shape, "parts:", parts.shape)

338024 881050
nodes: (338024, 2) edges: (881050, 3) parts: (338024, 12)


## Functions

In [4]:
#functions for data normalization
def toStd(data):
    return (data - mean_train) / std_train
def fromStd(data):
    return data * std_train + mean_train

# train, valid and infer
def run_epoch(model, sess, idxs, y, istrain=True, bs=4096): # Training Process
    loss, loss_step = [np.array([0.0]*3) for x in range(2)]
    st, ed, times, step = 0, bs, 0, 2000000
    time_step, times_old = step, 0
    while st < idxs.shape[0] and ed <= idxs.shape[0]:
        X_batch, y_batch = idxs[st:ed], y[st:ed]
        feed = {model.x_: X_batch, model.y_: toStd(y_batch)}
        if(istrain):
            loss_, loss_abs_, loss_rel_, _ = sess.run([model.loss, model.loss_abs, model.loss_rel, model.train_op], feed)
        else:
            loss_, loss_abs_, loss_rel_ = sess.run([model.loss_val, model.loss_abs_val, model.loss_rel_val], feed)
        loss += np.array([loss_, loss_abs_, loss_rel_])
        st, ed, times = ed, ed+bs, times+1
        if(times * bs >= time_step):
            loss_step_, loss_step[:] = loss - loss_step, loss[:]
            mean_step = np.mean(y[times_old*bs: times*bs])
            num_, times_old = times - times_old, times
            loss_step_ /= num_
            print("(%d): %.5f,%.5f"%(times * bs, loss_step_[1], loss_step_[2]))
            time_step += step
    return loss / times

def inference(model, sess, idxs): # Test Process
    start_time = time.time()
    pred = sess.run(model.pred_val, {model.x_: idxs})
    pred_ = fromStd(pred).astype(np.int)
    during_time = time.time() - start_time
    print("pred(%d)data: %d(mS)"%(pred_.shape[0], int(during_time*1000)))
    return pred_, np.mean(pred_)

# save/restore model
def save_emb(model, sess, fid):
    np.save(F_train_dir+"emb%08d"%fid, mlp_model.embedding_i.eval())
    print("model saved to " + F_train_dir+"emb%08d"%fid)
def load_emb(model, sess, fid):
    sess.run(mlp_model.emb_ass, feed_dict={mlp_model.emb_new: np.load(F_train_dir+"emb%08d.npy"%fid)})
def save_emb_int(model, fid, mid=0):
    emb_ = np.load(F_train_dir+"emb%08d.npy"%fid)
#     emb_ = mlp_model.embedding_i.eval()
    emb_ = fromStd(emb_) / F_man_dim
    save_csv(f_emb_output+str(mid), emb_.astype(np.int))
    print("save to ", f_emb_output+str(mid))

    
# map training samples into hier level
def getIdx(idxs, Transform=False, parts_=None):
    if(Transform): return parts_[idxs];
    else: return idxs;
    
def get_train_data(num, Transform, parts_, w):
    global F_n_train, idxs_val, y_val, idxs_train, y_train
    idx_ = np.random.choice(idxs_train.shape[0], idxs_train.shape[0], replace=False)
    idxs_train, y_train = idxs_train[idx_], y_train[idx_]
    idxs_train_, idxs_val_ = getIdx(idxs_train, Transform, parts_), getIdx(idxs_val, Transform, parts_)
    print("#training data: %d"%(y_train.shape[0]))
    return idxs_train_, y_train, idxs_val_, y_val

def train_epochs(sess, epochs, lr, bs, Transform=False, parts_=None, w=None, learning_rate_decay=True, checkpoint_id=0):
    pre_losses, best_val_loss, stay_cnt, last_loss = [1e18] * 3, 1e18, 0, 1e18
    lr_ass = mlp_model.learning_rate.assign(lr)
    sess.run(lr_ass)
    for epoch in range(epochs):
        start_time = time.time()
        idxs_train, y_train, idxs_val, y_val = get_train_data(F_n_train, Transform, parts_, w)
        
        print("Epoch(%d): lr= %f, bs= %d"%(epoch, mlp_model.learning_rate.eval(), bs))
        print("train: (#samples): mean abs err, mean abs rel err")
        train_loss = run_epoch(mlp_model, sess, idxs_train, y_train, istrain=True, bs=bs)
        print("valid:")
        val_loss = run_epoch(mlp_model, sess, idxs_val, y_val, istrain=False, bs=bs)

        if val_loss[0] <= best_val_loss:  # when valid_accuracy > best_valid_accuracy, save the model
            checkpoint_id += 1
            save_emb(mlp_model, sess, checkpoint_id)
            best_val_loss = val_loss[0]
            best_epoch = epoch + 1

        epoch_time = time.time() - start_time
        print("Epoch " + str(epoch) + " of " + str(epochs) + " took " + str(epoch_time) + "s")
        print("  training loss:             %.5f,%.5f"%tuple(train_loss[-2:]))
        print("  validation loss:           %.5f,%.5f"%tuple(val_loss[-2:]))
        print("")

        if(train_loss[0] >= last_loss):
            if(learning_rate_decay): stay_cnt += 1
            if(stay_cnt >= 3): sess.run(mlp_model.learning_rate_decay_op); stay_cnt = 0; last_loss = train_loss[0]
        else: stay_cnt = 0; last_loss = train_loss[0];

## Load Data

In [5]:
data_train = loadArrayInt(f_train)
idxs_train, y_train = data_train[:, :2], data_train[:, 2]
mean_train, std_train = np.mean(y_train), np.std(y_train)
mean_train, std_train = 0, mean_train

data_val = loadArrayInt(f_test)
idxs_val, y_val = data_val[:, :2], data_val[:, 2]
print(idxs_train.shape[0], idxs_val.shape[0], mean_train, std_train)

9500000 1000000 0 49960.952911157896


## Model

In [6]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tf.reset_default_graph()


Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
tf.reset_default_graph()
class Model:
    def __init__(self, learning_rate=0.04, learning_rate_decay_factor=0.8):
        self.y_ = tf.placeholder(tf.float32, [None])
        self.x_ = tf.placeholder(tf.int32, [None, 2])
        self.initializer_uniform = tf.initializers.random_uniform(-3/2, 3/2)
        
        self.loss, self.pred, self.loss_abs, self.loss_rel = self.forward(True)
        self.loss_val, self.pred_val, self.loss_abs_val, self.loss_rel_val = self.forward(False, reuse=True)

        self.learning_rate = tf.Variable(float(learning_rate), trainable=False, dtype=tf.float32)
        self.learning_rate_decay_op = self.learning_rate.assign(self.learning_rate * learning_rate_decay_factor)
        self.emb_new = tf.placeholder(tf.float32, shape=self.embedding_i.shape)
        self.emb_ass = self.embedding_i.assign(self.emb_new)

        self.params = tf.trainable_variables()
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)# update the BN params when training

        with tf.control_dependencies(update_ops):
            optimizer = tf.train.RMSPropOptimizer(self.learning_rate, 0.9, 0.9)
            gvs = optimizer.compute_gradients(self.loss)
            self.train_op = optimizer.apply_gradients(gvs)
        self.saver = tf.train.Saver(tf.global_variables(), write_version=tf.train.SaverDef.V2,
                                    max_to_keep=5, pad_step_number=True, keep_checkpoint_every_n_hours=1.0)
        print("model inited")
                                    
    def forward(self, is_train, reuse=None):
        with tf.variable_scope("model", reuse=reuse):
            self.embedding_i = tf.get_variable("embedding_i", [nodes.shape[0], F_man_dim], trainable=is_train, initializer=self.initializer_uniform)
            xs_em = tf.nn.embedding_lookup(self.embedding_i, self.x_)#shape: (bs, 2, l_emb)
            if(is_train): print(xs_em.shape)
            dx_ = tf.abs(xs_em[:, 0, :] - xs_em[:, 1, :])# emb1 - emb2
            pred = tf.reduce_mean(dx_, axis=1) #expect when w=1, pred and y_ are in the same level

        if(is_train): print(pred.shape)
        df_ = pred - self.y_
        loss, loss_abs = tf.reduce_mean(tf.square(df_)), tf.reduce_mean(tf.abs(df_))
        
        y_, pred_ = fromStd(self.y_), fromStd(pred)
        diff_abs_ = tf.abs(pred_ - y_)
        diff_abs = tf.reduce_mean(diff_abs_)
        diff_rel = tf.reduce_mean(diff_abs_ / tf.maximum(y_, 1.0))#(y_ + F_rel_fac))
        return loss, pred, diff_abs, diff_rel

mlp_model = Model()

(?, 2, 64)
(?,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
model inited


## Hierarchical training

In [8]:
# check #nodes in level $lev
lev = 8
np.max(partDict(parts[:, lev])) + 1

np.int64(66601)

In [9]:
n_epoch, n_hier, ptsIdxs = 10, 6, np.array([4, 5, 6, 7, 8, 11])
epochs, bss, lrs = np.array([1, 1, 1, 1, 1, n_epoch]), np.array([F_batch_size]*n_hier), np.array([F_learning_rate]*n_hier)
embeddings = np.zeros((nodes.shape[0], F_man_dim))
F_train_version = 0

In [10]:
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
    if(F_train_version == 0):
        tf.global_variables_initializer().run()
    else:
        restore_model(mlp_model, sess, F_train_version)
    for idx in range(0, n_hier):
        parts_ = partDict(parts[:, ptsIdxs[idx]])
        n_p = np.max(parts_) + 1
        print("level %d: %d nodes"%(idx, n_p))
        # input embs to model
        if(idx != 0):#assign new embs for next training(prolongation)
            parts__ = partDict(parts[:, ptsIdxs[idx-1]])
            embeddings[parts_] = mlp_model.embedding_i.eval()[parts__]
            sess.run(mlp_model.emb_ass, feed_dict={mlp_model.emb_new: embeddings})
        #train...
        train_epochs(sess, epochs[idx], lrs[idx], bss[idx], True, parts_, w=None, learning_rate_decay=False, checkpoint_id=F_train_version)

E0000 00:00:1761195329.213098 3572790 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
E0000 00:00:1761195329.214923 3572790 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1761195329.222292 3572790 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
I0000 00:00:1761195329.231964 3572790 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled


level 0: 256 nodes
#training data: 9500000
Epoch(0): lr= 0.040000, bs= 4096
train: (#samples): mean abs err, mean abs rel err
(2002944): 10387.41596,0.38667
(4001792): 3811.87694,0.12098
(6000640): 3813.27620,0.12083
(8003584): 3811.14987,0.12058
valid:
model saved to ./train/BJ/model/emb00000001
Epoch 0 of 1 took 8.459240913391113s
  training loss:             5198.27059,0.17680
  validation loss:           3800.50007,0.12088

level 1: 1024 nodes
#training data: 9500000
Epoch(0): lr= 0.040000, bs= 4096
train: (#samples): mean abs err, mean abs rel err
(2002944): 2537.66886,0.08298
(4001792): 2158.47081,0.07058
(6000640): 2147.82021,0.07038
(8003584): 2143.42515,0.06984
valid:
model saved to ./train/BJ/model/emb00000001
Epoch 0 of 1 took 10.170874118804932s
  training loss:             2230.90003,0.07292
  validation loss:           2156.70360,0.07085

level 2: 4096 nodes
#training data: 9500000
Epoch(0): lr= 0.040000, bs= 4096
train: (#samples): mean abs err, mean abs rel err
(2002944

## Train or Test

In [11]:
F_num_epochs = 100
if not hasattr(np, "int"):
    np.int = int
if not hasattr(np, "float"):
    np.float = float
if not hasattr(np, "bool"):
    np.bool = bool
F_is_train, F_reuse_model, F_train_version, F_inference_version = 0, 0, 0, 2
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))) as sess:
    if F_is_train:
        if F_train_version == 0:
            restore_model(mlp_model, sess, F_train_version)
            lr_op = mlp_model.learning_rate.assign(F_learning_rate)
            sess.run(lr_op)
        else:
            sess.run(tf.global_variables_initializer())

        train_epochs(
            sess, F_num_epochs, F_learning_rate, F_batch_size,
            False, checkpoint_id=F_train_version if F_reuse_model else 0
        )

    else:
        # 推理模式
        load_emb(mlp_model, sess, F_inference_version)
        idxs_test, y_test = idxs_val, y_val

        pred, avr = inference(mlp_model, sess, idxs_test)
        diff_ = np.abs(pred - y_test)
        diff_rel = diff_ / np.maximum(y_test, 1)

        print("    abs:", np.mean(diff_))
        print("    rel:", np.mean(diff_rel))
        print("rel std:", np.std(diff_rel))

E0000 00:00:1761195610.355788 3572790 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
E0000 00:00:1761195610.357712 3572790 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1761195610.361580 3572790 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


pred(1000000)data: 98(mS)
    abs: 407.388561
    rel: 0.016208767798120263
rel std: 0.13756290234125598
